<a href="https://colab.research.google.com/github/mwolff615/COVID-19/blob/master/Assignment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install plotly==4.6.0
!pip install plotly_express==0.4.1
!pip install ipynb

In [46]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
import plotly.graph_objects as go
import numpy as np
import plotly.graph_objects as go

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [47]:
df = pd.read_csv("/content/drive/My Drive/VIZ/percentcovid.csv")
state = pd.read_csv("/content/drive/My Drive/VIZ/stateabb.csv")
locList = state['abb'].tolist()
percList = df['prop'].tolist()
y = []
for x in percList:
  y.append(np.log(x))
fig = px.choropleth(locations = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
, locationmode = 'USA-states', color = y, scope = 'usa') 
fig.show()

In [0]:
df1 = pd.read_csv('/content/drive/My Drive/DescartesData/DL-us-m50.csv', dtype={"fips": str})
df1 = df1.dropna(subset = ['admin2'])
df1 = df1.dropna(axis = 0)
df2 = pd.read_csv('/content/drive/My Drive/DescartesData/DL-us-m50_index.csv')
df3 = pd.read_csv('/content/drive/My Drive/DescartesData/DL-us-mobility-daterow.csv')
df4 = pd.read_csv('/content/drive/My Drive/DescartesData/DL-us-samples.csv')
df1 = df1[df1.fips != '35037']
df1 = df1[df1.fips != '35006']

In [0]:
def create_Choropleth(toGraph, zmax = 5, zmin = -5, colorscale = 'viridis', saveFig = False): 
  fig = go.Figure(go.Choroplethmapbox(geojson=counties, locations=df1.fips,z = np.log(df1[x]), zmax = 5, zmin = -5, colorscale  = 'viridis',
                                      marker_opacity=0.5, marker_line_width=0))
  fig.update_layout(mapbox_style="carto-positron",
                    mapbox_zoom=3, mapbox_center = {"lat": 37.0902, "lon": -95.7129})
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.show()
  matplotlib.pyplot.savefig(x + ".png")
  #for x in datelist:


In [50]:
def stockMarketPredictions(data, past = 30, p = 2, d = 2, q = 2): 
    from statsmodels.tsa.arima_model import ARIMA, ARIMAResults
    data0 = pd.read_csv(data)
    close0 = data0['close'].tolist()
    closeFull = close0
    closeSubset = close0[0:past]
    min_aic = 10000
    storedModel = None

    for x in range(0, p): 
        for y in range(0, d):
            for z in range(0, q):
                try:
                    model = ARIMA(closeSubset, order = (x, y, z), missing = "drop")
                    model_fit = model.fit() 
                    if(model_fit.aic < min_aic): 
                        min_aic = model_fit.aic
                        storedModel = model_fit
                except:
                    pass
    return storedModel; 
def getReality(data): 
    data0 = pd.read_csv(data)
    close0 = data0['close'].tolist()
    return close0; 
def makePredictions(model, future = 5): 
    return model.forecast(future)[0]
def plotPredictions(predictions, reality, stock, past = 30, future = 5): 
    plt.style.use('dark_background')
    fig, ax1 = plt.subplots()
    plt.plot(reality[past:past+future])
    plt.plot(predictions)
    ax1.set_xlabel('Future Projections (Days)')
    ax1.set_title('Real and Predicted\nStock Market Closing Values\n[' + stock +']')
    ax1.set_ylabel('Closing Values')
    plt.show()
'''predictions = makePredictions(stockMarketPredictions('ACN_data.csv'))
pred2 = makePredictions(stockMarketPredictions('ABT_data.csv'))
plotPredictions(predictions, getReality('ACN_data.csv'), 'ACN')
plotPredictions(pred2, getReality('ABT_data.csv'), 'ABT')'''

FileNotFoundError: ignored

In [0]:
#resources
#https://stackoverflow.com/questions/44549110/python-loop-through-excel-sheets-place-into-one-df
#https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory
#https://stackoverflow.com/questions/40835287/python-error-tokenizing-data-c-error-calling-readnbytes-on-source-failed-wi
#https://www.google.com/search?client=firefox-b-1-d&q=pass+vs+continue+python
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
#https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
#https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
#https://www.geeksforgeeks.org/python-pandas-dataframe-append/
#https://datatofish.com/pandas-dataframe-to-sql/
#https://www.google.com/search?client=firefox-b-1-d&q=date+field+sqlite+3
#https://datatofish.com/pandas-dataframe-to-sql/
#https://www.geeksforgeeks.org/how-to-rename-columns-in-pandas-dataframe/
#https://matplotlib.org/3.2.1/gallery/lines_bars_and_markers/barchart.html#sphx-glr-gallery-lines-bars-and-markers-barchart-py
#https://nssac.bii.virginia.edu/covid-19/dashboard/
